In [42]:
import os
import re

import cv2
import matplotlib.pyplot as plt

import mgitools.os_helpers as os_helpers

In [43]:
images_folder = '/Users/erikstorrs/Downloads/BBBC020_v1_images/'
masks_folder = '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/'

##### grab combined images

In [44]:
images_fps = list(os_helpers.listfiles(images_folder, regex=r'c1.*c5'))

sample_id_to_images_fp = {}
for fp in images_fps:
    pieces = fp.split('/')
    
    sample_id = pieces[-2].replace(' ', '-')
    
    sample_id_to_images_fp[sample_id] = fp

In [45]:
len(sample_id_to_images_fp), list(sample_id_to_images_fp.items())[:5]

(25,
 [('jw-1h-2',
   '/Users/erikstorrs/Downloads/BBBC020_v1_images/jw-1h 2/jw-1h 2_(c1+c5).TIF'),
  ('jw-1h-5',
   '/Users/erikstorrs/Downloads/BBBC020_v1_images/jw-1h 5/jw-1h 5_(c1+c5).TIF'),
  ('jw-1h-4',
   '/Users/erikstorrs/Downloads/BBBC020_v1_images/jw-1h 4/jw-1h 4_(c1+c5).TIF'),
  ('jw-1h-3',
   '/Users/erikstorrs/Downloads/BBBC020_v1_images/jw-1h 3/jw-1h 3_(c1+c5).TIF'),
  ('jw-2h-1',
   '/Users/erikstorrs/Downloads/BBBC020_v1_images/jw-2h 1/jw-2h 1_(c1+c5).TIF')])

##### grab masks

In [46]:
masks_fps = list(os_helpers.listfiles(masks_folder, regex=r'TIF'))

sample_id_to_mask_fps = {}
for fp in masks_fps:
    sample_id = re.sub(r'^.*outlines_cells/(.*)_c1.*TIF$', r'\1', fp).replace(' ', '-')
    
    if sample_id in sample_id_to_mask_fps:
        sample_id_to_mask_fps[sample_id].append(fp)
    else:
        sample_id_to_mask_fps[sample_id] = [fp]

In [47]:
len(sample_id_to_mask_fps), list(sample_id_to_mask_fps.items())[:1]

(20,
 [('jw-24h-2',
   ['/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_3.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_12.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_13.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_2.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_0.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_11.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_10.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_1.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_5.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_14.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_15.TIF',
    '/Users/erikstorrs/Downloads/BBC020_v1_outlines_cells/jw-24h 2_c1_4.TIF',
    '/Users/erikstorrs/Downloads/BBC02

##### convert to png and save in correct order

In [48]:
output_dir = '/Users/erikstorrs/Documents/ding/cell_segmentation/data/BBC020'

In [49]:
os.mkdir(output_dir)

FileExistsError: [Errno 17] File exists: '/Users/erikstorrs/Documents/ding/cell_segmentation/data/BBC020'

In [60]:
for sample_id, image_fp in sample_id_to_images_fp.items():
    if sample_id in sample_id_to_mask_fps:
        os.mkdir(os.path.join(output_dir, sample_id))
        os.mkdir(os.path.join(output_dir, sample_id, 'images'))

        image = cv2.imread(image_fp, flags=cv2.IMREAD_COLOR)
        image = cv2.resize(image, (0,0), fx=0.25, fy=0.25) 
        
        cv2.imwrite(os.path.join(output_dir, sample_id, 'images', f'{sample_id}.png'), image)

In [61]:
for sample_id, mask_fps in sample_id_to_mask_fps.items():
    os.mkdir(os.path.join(output_dir, sample_id, 'masks'))
    
    for i, image_fp in enumerate(mask_fps):
        image = cv2.imread(image_fp, flags=cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (0,0), fx=0.25, fy=0.25)
        
        cv2.imwrite(os.path.join(output_dir, sample_id, 'masks', f'{sample_id}_{i}.png'), image)